In [ ]:
%pip install pymysql
%pip install sshtunnel
%pip install pandas

In [3]:
import pymysql
from sshtunnel import SSHTunnelForwarder
import paramiko
from os.path import expanduser
import pandas as pd

In [4]:
import numpy as np

In [2]:
import tensorflow as tf
print(tf.__version__)

Init Plugin
2.5.0
Init Graph Optimizer
Init Kernel


In [19]:
import time

In [27]:
home = expanduser('~')
mypkey = paramiko.RSAKey.from_private_key_file( home + "/.ssh/id_rsa")

sql_hostname = '127.0.0.1'
sql_username = 'readonly'
sql_password = None
sql_main_database = 'ML3_mirror'
sql_port = 3306
ssh_host = 'flagon.cs.umn.edu'
ssh_user = 'kanna128'
ssh_port = 22
sql_ip = '1.1.1.1.1'

def table1_data():
  with SSHTunnelForwarder(
          (ssh_host, ssh_port),
          ssh_username=ssh_user,
          ssh_pkey=mypkey,
          remote_bind_address=(sql_hostname, sql_port)) as tunnel:
      conn = pymysql.connect(host='127.0.0.1', user=sql_username,
                              passwd=sql_password, db=sql_main_database,
                              port=tunnel.local_bind_port)
      print('connection done')

      # query to calculate the number of users
      query_users = ''' SELECT expt_user.group, expt_user.userId from ML3_mirror.expt_user WHERE 
                        exptId = 'listcmp-long' AND started <'2015-04-01' 
                        AND status = 'RANDOMLY_ASSIGNED' '''
      users = pd.read_sql_query(query_users, conn)
      users_count = users.count()['group']
      print("Users: {}".format(users_count))

      recchange_dict = {} # dict to store key=userId, val=curralg in the iteration process
      initial_alg = {}

      # key=userId; val=number of changes
      changes_dict = {} # needed to later calculate median changes per user
      for k in range(users_count):
        userId = users.at[k, 'userId']
        group = "{\"engineId\":\""+users.at[k, 'group']+"\"}" # following the format of log_action.log_Json for future comparisons
        recchange_dict[userId] = group
        initial_alg[userId] = group
        changes_dict[userId] = 0

      # query to calculate the number of recommender change events
      # note: adding/ removing started<'2015-04-01' inside the EXISTS(...) part doesn't change the count
      # avoid variable separation
      # ORDER BY: to sort by timestamp

      # note: ORDER BY tstamp doesn't change results as of now
      query_change = '''SELECT userId, logJson, tstamp, action from ML3_mirror.log_action WHERE EXISTS 
                        (SELECT userId from ML3_mirror.expt_user where expt_user.userId = log_action.userId 
                        AND expt_user.exptId='listcmp-long' 
                        AND expt_user.status='RANDOMLY_ASSIGNED') 
                        AND log_action.action='recommender-change' 
                        AND log_action.tstamp<'2015-04-01' ORDER BY log_action.tstamp'''
      change_events = pd.read_sql_query(query_change, conn)
      print("query done")
      change_events.sort_values(by=['userId'])

      length = change_events.count()['userId']
      
      for i in range(length):
        userId = change_events.at[i, 'userId']
        recalg = change_events.at[i, 'logJson']
        if(recchange_dict[userId] != recalg):
          changes_dict[userId] += 1
        
        recchange_dict[userId] = recalg
      
      changes_dict = {key:val for key,val in changes_dict.items() if val !=0}

      print("Fig3_basic start")
      baseline = 0
      item_item = 0
      svd = 0

      count_dict = {"{\"engineId\":\"baseline\"}":0, "{\"engineId\":\"item-item\"}":0, "{\"engineId\":\"svd\"}":0, "{\"engineId\":\"pick-groups\"}":0}
      for userId,recalg in initial_alg.items():
        count_dict[recalg] +=1
        if userId in changes_dict.keys():
          if(recalg == "{\"engineId\":\"baseline\"}"):
            baseline += 1
          if(recalg == "{\"engineId\":\"item-item\"}"):
            item_item += 1
          if(recalg == "{\"engineId\":\"svd\"}"):
            svd += 1

      baseline = baseline/count_dict["{\"engineId\":\"baseline\"}"]
      item_item = item_item/count_dict["{\"engineId\":\"item-item\"}"]
      svd = svd/count_dict["{\"engineId\":\"svd\"}"]

      print("proportion of users who switched from baseline: ", baseline)
      print("proportion of users who switched from item_item: ", item_item)
      print("proportion of users who switched from svd: ", svd)

      print("Fig3_basic done")

      
      # repeated conditional statements -- are they okay?
      # alternative: dictionary --> key=string to compare, val=variable to increment
      # tradeoff: additional memory usage 
      # the tradeoff probably doesn't matter much
      # an immediate access mechanism to increment variables is better than a series of if statements holding the loop back for a while 
      # time this part of the program to experiment with these ideas

      for key in count_dict.keys():
        count_dict[key] = 0

      print(count_dict["{\"engineId\":\"baseline\"}"])

      # start_time = time.time()
      for userId, recalg in recchange_dict.items():
        if(userId in changes_dict.keys()):
          count_dict[recalg] += 1
          # if(recalg == "{\"engineId\":\"baseline\"}"): # format of string to compare with is the same as format of log_action.logJson
          #   baseline += 1
          # if(recalg == "{\"engineId\":\"item-item\"}"):
          #   item_item += 1
          # if(recalg == "{\"engineId\":\"pick-groups\"}"):
          #   pick_groups += 1
          # if(recalg == "{\"engineId\":\"svd\"}"):
          #   svd += 1
      # print("seconds: ", time.time() - start_time)

      print("Recommender change events: ", len(changes_dict.items()))
      print("Users switching at least once: ", len(changes_dict.values()))
      print("Median changes per user w/ at least 1 change: ", int(np.median(list(changes_dict.values()))))
      print("Baseline: ", count_dict["{\"engineId\":\"baseline\"}"])
      print("Item-item: ", count_dict["{\"engineId\":\"item-item\"}"])
      print("Pick Groups: ", count_dict["{\"engineId\":\"pick-groups\"}"])
      print("Svd: ", count_dict["{\"engineId\":\"svd\"}"])

      # expt_end_date - account_creation_date as a count of days

if __name__ == "__main__":
  table1_data()
  print('done')

connection done
Users: 3005
query done
Recommender change events:  748
Fig3_basic start
proportion of users who switched from baseline:  0.296875
proportion of users who switched from item_item:  0.2469879518072289
proportion of users who switched from svd:  0.20101522842639594
Fig3_basic done
0
Recommender change events:  748
Users switching at least once:  748
Median changes per user w/ at least 1 change:  3
Baseline:  54
Item-item:  290
Pick Groups:  62
Svd:  342
done


,Unnamed: 0,userId,logJson,tstamp,action
0,0,84090,"{""engineId"":""pick-groups""}",2014-10-31 15:22:45,recommender-change
1,1,84090,"{""engineId"":""baseline""}",2014-10-31 15:23:13,recommender-change
2,2,84090,"{""engineId"":""pick-groups""}",2014-10-31 15:23:37,recommender-change
3,3,84090,"{""engineId"":""baseline""}",2014-10-31 15:23:46,recommender-change
4,4,84090,"{""engineId"":""pick-groups""}",2014-10-31 15:23:53,recommender-change
